In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the cleaned data
df = pd.read_csv('../data/processed/fight_data_cleaned.csv')

print("ONE PIECE FIGHT PREDICTOR - MODEL TRAINING")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['outcome'].value_counts()}")

ONE PIECE FIGHT PREDICTOR - MODEL TRAINING
Dataset shape: (4950, 46)
Target distribution:
outcome
victory    2849
loss       2027
draw         74
Name: count, dtype: int64


In [2]:
# Data Preparation
print("\nDATA PREPARATION")
print("=" * 30)

# 1. Examine the dataset structure
print("Dataset columns:")
print(df.columns.tolist())
print(f"\nDataset info:")
print(f"- Total fights: {len(df)}")
print(f"- Features: {len(df.columns) - 1}")  # excluding outcome
print(f"- Missing values: {df.isnull().sum().sum()}")

# 2. Prepare features and target
# Remove non-numeric columns (fight names if they exist)
# Debug: Check what columns we actually have
print("All columns in dataset:")
print(df.columns.tolist())
print("\nColumn data types:")
print(df.dtypes)

# Check for any string columns
print("\nString/object columns:")
string_cols = df.select_dtypes(include=['object']).columns.tolist()
print(string_cols)

# Fix the feature selection - exclude ALL non-numeric columns
feature_cols = [col for col in df.columns if col not in ['outcome'] and df[col].dtype in ['int64', 'float64']]

print(f"\nCorrected feature columns ({len(feature_cols)}):")
for i, col in enumerate(feature_cols, 1):
    print(f"{i:2d}. {col}")

# Update X with only numeric features
X = df[feature_cols]
y = df['outcome']

print(f"\nNew X shape: {X.shape}")
print(f"All columns are numeric: {X.dtypes.apply(lambda x: x.name in ['int64', 'float64']).all()}")

# 3. Encode target variable
print(f"\nTarget Variable Encoding:")
print(f"Original outcome distribution:")
print(y.value_counts())

# Encode all outcomes as separate classes (including draws)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\nEncoded target:")
print(f"Classes: {le.classes_}")
print(f"Encoded distribution:")
for i, class_name in enumerate(le.classes_):
    count = (y_encoded == i).sum()
    print(f"  {class_name}: {count}")

# Check if we have 3 classes (victory, loss, draw)
n_classes = len(le.classes_)
print(f"\nClassification type: {'Multi-class' if n_classes > 2 else 'Binary'} ({n_classes} classes)")

# 4. Split the data
print(f"\nTRAIN-TEST SPLIT")
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_encoded  # Maintain class distribution
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Feature count: {X_train.shape[1]}")

# 5. Feature scaling
print(f"\nFEATURE SCALING")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features scaled using StandardScaler")
print(f"Training features mean: {X_train_scaled.mean():.6f}")
print(f"Training features std: {X_train_scaled.std():.6f}")

# 6. Data summary
print(f"\nFINAL DATASET SUMMARY")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"Features: {X_train.shape[1]}")
print(f"Classes: {len(le.classes_)} ({le.classes_})")
print(f"Class distribution in training:")
for i, class_name in enumerate(le.classes_):
    count = (y_train == i).sum()
    pct = count / len(y_train) * 100
    print(f"  {class_name}: {count} ({pct:.1f}%)")


DATA PREPARATION
Dataset columns:
['fight_name', 'fighter_1_reaction_speed', 'fighter_2_reaction_speed', 'reaction_speed_diff', 'reaction_speed_advantage', 'fighter_1_stamina', 'fighter_2_stamina', 'stamina_diff', 'stamina_advantage', 'fighter_1_strength', 'fighter_2_strength', 'strength_diff', 'strength_advantage', 'fighter_1_offense', 'fighter_2_offense', 'offense_diff', 'offense_advantage', 'fighter_1_defense', 'fighter_2_defense', 'defense_diff', 'defense_advantage', 'fighter_1_combat_skills', 'fighter_2_combat_skills', 'combat_skills_diff', 'combat_skills_advantage', 'fighter_1_battle_iq', 'fighter_2_battle_iq', 'battle_iq_diff', 'battle_iq_advantage', 'fighter_1_armament_haki', 'fighter_2_armament_haki', 'armament_haki_diff', 'armament_haki_advantage', 'fighter_1_observation_haki', 'fighter_2_observation_haki', 'observation_haki_diff', 'observation_haki_advantage', 'fighter_1_conqueror_haki', 'fighter_2_conqueror_haki', 'conqueror_haki_diff', 'conqueror_haki_advantage', 'fighter

### Attribute Variants Analysis:
For each 1 core attribute (e.g., strength), we have 4 variants:
- fighter_1_strength (raw val for fighter 1)
- fighter_2_strength (raw val for fighter 2)
- strength_diff (fighter_1 - fighter_2)
- strength_advantage (1 if fighter_1 > fighter_2, else 0)

### Options:

1. Use ALL variants:
- **Pros**: maximum information
- **Cons**: redundancy, multicollinearity, overfitting risk

2. Use only DIFF variables:
- **Pros**: captures relative strength directly, less redundant
- **Cons**: loses absolute scale information (e.g., 10 - 8 is the same as 3 - 1)

3. Use only ADVANTAGE variables:
- **Pros**: simomple binary features, interpretable
- **Cons**: loses magnitude information 

4. Use DIFF + 1 raw value:
- **Pros**: relative comparison + absolute scale context
- **Cons**: still some redundancy

### Final Decision: Option 2
- **Fight prediction is inherently relative**: what matters is who's stronger, not absolute power levels
- **EDA**: _diff and _advantage variables had the highest correlations with outcome
- **Multicollinearity**: reduced significantly
- **Simpler model**: fewer features but same predictive power

### Proposed Feature Set: 
reaction_speed_diff, stamina_diff, strength_diff, offense_diff, defense_diff,combat_skills_diff, battle_iq_diff, armament_haki_diff, observation_haki_diff, conqueror_haki_diff, experience_diff

In [5]:
# Feature Selection: Use only DIFFERENCE variables
print("\nFEATURE SELECTION: DIFFERENCE VARIABLES ONLY")
print("=" * 50)

# Define the 11 core difference variables
diff_features = [
    'reaction_speed_diff',
    'stamina_diff', 
    'strength_diff',
    'offense_diff',
    'defense_diff',
    'combat_skills_diff',
    'battle_iq_diff',
    'armament_haki_diff',
    'observation_haki_diff',
    'conqueror_haki_diff',
    'experience_diff'
]

print(f"Selected difference features ({len(diff_features)}):")
for i, feature in enumerate(diff_features, 1):
    print(f"{i:2d}. {feature}")

# Check if all features exist in the dataset
missing_features = [f for f in diff_features if f not in df.columns]
if missing_features:
    print(f"\nMissing features: {missing_features}")
else:
    print(f"\nAll {len(diff_features)} features found in dataset")

# Update X with only the difference variables
X = df[diff_features]
y = df['outcome']

print(f"\nUpdated dataset:")
print(f"- Features: {X.shape[1]}")
print(f"- Samples: {X.shape[0]}")
print(f"- All numeric: {X.dtypes.apply(lambda x: x.name in ['int64', 'float64']).all()}")

# Quick correlation check with outcome
y_encoded_temp = LabelEncoder().fit_transform(y)
print(f"\nAll correlations with outcome:")
correlations = []
for feature in diff_features:
    corr = np.corrcoef(X[feature], y_encoded_temp)[0, 1]
    correlations.append((feature, abs(corr)))

correlations.sort(key=lambda x: x[1], reverse=True)
for feature, corr in correlations:
    print(f"  {feature}: {corr:.3f}")


FEATURE SELECTION: DIFFERENCE VARIABLES ONLY
Selected difference features (11):
 1. reaction_speed_diff
 2. stamina_diff
 3. strength_diff
 4. offense_diff
 5. defense_diff
 6. combat_skills_diff
 7. battle_iq_diff
 8. armament_haki_diff
 9. observation_haki_diff
10. conqueror_haki_diff
11. experience_diff

All 11 features found in dataset

Updated dataset:
- Features: 11
- Samples: 4950
- All numeric: True

All correlations with outcome:
  strength_diff: 0.720
  reaction_speed_diff: 0.704
  stamina_diff: 0.701
  combat_skills_diff: 0.697
  offense_diff: 0.690
  defense_diff: 0.677
  armament_haki_diff: 0.676
  battle_iq_diff: 0.656
  experience_diff: 0.650
  observation_haki_diff: 0.626
  conqueror_haki_diff: 0.414


### Potential Solutions for the Conqueror's Haki dilemma

1. 2 Separate models:
- Approach: train 1 model for conqueror fights, 1 for non-conqueror fights
- Pros:
    - Captures true conqueror impact in relevant fights
    - Each model optimized for its specific context
- Cons:
    - Need to maintain 2 models
    - Conqueror model has limited training data
    - Deployment complexity (need to check whether fighters possess Conqueror first)

2. Feature Engineering:
- Approach: create interaction terms
    - **conqueror_present**: binary (1 if either fighter has it)
    - **conqueror_haki_diff** * **conqueror_present**: amplified difference when relevance
- Pros:
    - Single model
    - Captures conditional importance
- Cons:
    - More complex feature engineering
    - Model needs to learn the interaction

3. Stratified Approach:
- Approach: keep single model but weigh Conqueror fights more heavily during training
- Pros:
    - Single model
    - Emphasizes important cases
- Cons: may bias toward conqeuror scenarios

4. Ensemble Approach:
- Approach: train both specialized models + combine predictions
- Pros:
    - best of both worlds
    - can weigh predictions based on confidence
- Cons: most complex implementation

### Final Decision: Option 2 (Feature Engineering)
- **Practical deployment**: single model is easier to maintain
- **Data efficiency**: uses all data for training
- **Captures interaction**: model learns when conqueror matters
- **Interpretable**: clear features show the conditional effect

In [ ]:
# Feature Engineering: Handle Conqueror's Haki interaction
print("\nFEATURE ENGINEERING: CONQUEROR'S HAKI INTERACTION")
print("=" * 60)

# Start with the base difference features (removing conqueror_haki_diff for now)
base_diff_features = [
    'reaction_speed_diff',
    'stamina_diff', 
    'strength_diff',
    'offense_diff',
    'defense_diff',
    'combat_skills_diff',
    'battle_iq_diff',
    'armament_haki_diff',
    'observation_haki_diff',
    'experience_diff'
]

# Create the engineered features
print("Creating Conqueror's Haki interaction features...")

# 1. conqueror_present: 1 if either fighter has Conqueror's Haki
df['conqueror_present'] = ((df['fighter_1_conqueror_haki'] > 0) | 
                          (df['fighter_2_conqueror_haki'] > 0)).astype(int)

# 2. conqueror_impact: amplified difference when relevant
df['conqueror_impact'] = df['conqueror_haki_diff'] * df['conqueror_present']

# Final feature set: 10 base + 2 engineered = 12 features
final_features = base_diff_features + ['conqueror_present', 'conqueror_impact']

print(f"\nFinal feature set ({len(final_features)} features):")
for i, feature in enumerate(final_features, 1):
    print(f"{i:2d}. {feature}")

# Update X with the engineered features
X = df[final_features]
y = df['outcome']

print(f"\nUpdated dataset:")
print(f"- Features: {X.shape[1]}")
print(f"- Samples: {X.shape[0]}")

# Analyze the new features
print(f"\nConqueror's Haki Feature Analysis:")
print(f"Fights with Conqueror present: {df['conqueror_present'].sum()}")
print(f"Percentage: {df['conqueror_present'].mean()*100:.1f}%")

print(f"\nConqueror impact statistics:")
print(f"- Mean: {df['conqueror_impact'].mean():.3f}")
print(f"- Std: {df['conqueror_impact'].std():.3f}")
print(f"- Non-zero values: {(df['conqueror_impact'] != 0).sum()}")

# Check correlations with the new features
y_encoded_temp = LabelEncoder().fit_transform(y)
correlations = []
for feature in final_features:
    corr = np.corrcoef(X[feature], y_encoded_temp)[0, 1]
    correlations.append((feature, abs(corr)))

correlations.sort(key=lambda x: x[1], reverse=True)
print(f"\nTop correlations with outcome (engineered features):")
for feature, corr in correlations:
    marker = "🔥" if "conqueror" in feature else "  "
    print(f"{marker} {feature}: {corr:.3f}")


FEATURE ENGINEERING: CONQUEROR'S HAKI INTERACTION
Creating Conqueror's Haki interaction features...

Final feature set (12 features):
 1. reaction_speed_diff
 2. stamina_diff
 3. strength_diff
 4. offense_diff
 5. defense_diff
 6. combat_skills_diff
 7. battle_iq_diff
 8. armament_haki_diff
 9. observation_haki_diff
10. experience_diff
11. conqueror_present
12. conqueror_impact

Updated dataset:
- Features: 12
- Samples: 4950

Conqueror's Haki Feature Analysis:
Fights with Conqueror present: 1464
Percentage: 29.6%

Conqueror impact statistics:
- Mean: 0.181
- Std: 0.489
- Non-zero values: 1344

🎯 Top correlations with outcome (engineered features):
   strength_diff: 0.720
   reaction_speed_diff: 0.704
   stamina_diff: 0.701
   combat_skills_diff: 0.697
   offense_diff: 0.690
   defense_diff: 0.677
   armament_haki_diff: 0.676
   battle_iq_diff: 0.656
   experience_diff: 0.650
   observation_haki_diff: 0.626
🔥 conqueror_impact: 0.414
🔥 conqueror_present: 0.224


In [7]:
# Verify the feature engineering is working
print("\nVERIFYING CONQUEROR'S HAKI FEATURE ENGINEERING")
print("=" * 60)

# Compare correlations in different fight types
conqueror_fights_mask = df['conqueror_present'] == 1
non_conqueror_fights_mask = df['conqueror_present'] == 0

print(f"Total fights: {len(df)}")
print(f"Conqueror fights: {conqueror_fights_mask.sum()}")
print(f"Non-Conqueror fights: {non_conqueror_fights_mask.sum()}")

# Correlations in Conqueror fights only
conqueror_fights = df[conqueror_fights_mask]
if len(conqueror_fights) > 0:
    y_conqueror = LabelEncoder().fit_transform(conqueror_fights['outcome'])
    
    # Original conqueror_haki_diff correlation
    corr_original = np.corrcoef(conqueror_fights['conqueror_haki_diff'], y_conqueror)[0, 1]
    
    # New conqueror_impact correlation (should be identical in these fights)
    corr_impact = np.corrcoef(conqueror_fights['conqueror_impact'], y_conqueror)[0, 1]
    
    print(f"\nCorrelations in CONQUEROR fights only:")
    print(f"  Original conqueror_haki_diff: {abs(corr_original):.3f}")
    print(f"  New conqueror_impact: {abs(corr_impact):.3f}")
    print(f"  Difference: {abs(corr_impact - corr_original):.6f} (should be ~0)")

# The key insight: Model can now learn the conditional relationship
print(f"\nTHE KEY INSIGHT:")
print(f"The model can now learn:")
print(f"- When conqueror_present = 1: conqueror_impact matters a lot")
print(f"- When conqueror_present = 0: conqueror_impact is always 0 (irrelevant)")
print(f"This is better than raw conqueror_haki_diff which was always 0 for most fights")


VERIFYING CONQUEROR'S HAKI FEATURE ENGINEERING
Total fights: 4950
Conqueror fights: 1464
Non-Conqueror fights: 3486

Correlations in CONQUEROR fights only:
  Original conqueror_haki_diff: 0.737
  New conqueror_impact: 0.737
  Difference: 0.000000 (should be ~0)

THE KEY INSIGHT:
The model can now learn:
- When conqueror_present = 1: conqueror_impact matters a lot
- When conqueror_present = 0: conqueror_impact is always 0 (irrelevant)
This is better than raw conqueror_haki_diff which was always 0 for most fights
